In [ ]:
import numpy as np 
import cv2 
import pandas as pd
import glob
import time 
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
download_path='C:/Users/IV/Desktop/CityU' # change to your own download path
path_query=download_path+'/query_4186'
path_query_txt=download_path+'/query_txt_4186'

# path_query_txt is the directory to the bounding box information of the instance(s) for the query images
path_gallery=download_path+'/gallery_4186'

name_query=glob.glob(path_query+'/*.jpg')
num_query=len(name_query)
name_gallery=glob.glob(path_gallery+'/*.jpg')
num_gallery=len(name_gallery)

sift = cv2.ORB_create()
bf = cv2.BFMatcher()

record_all=np.zeros((num_query,len(name_gallery)))

query_imgs_no = [x.split('\\')[-1] for x in glob.glob(path_query+'/*.jpg')]
query_imgs_no = [x[:-4] for x in query_imgs_no]

gallery_imgs_no = [x.split('\\')[-1] for x in glob.glob(path_gallery+'/*.jpg')]
gallery_imgs_no = [x[:-4] for x in gallery_imgs_no]

gallery_kp = []
gallery_des = []

for i, gallery_img_no in tqdm(enumerate(gallery_imgs_no)):
    per_gallery_name = path_gallery+'/'+str(gallery_img_no)+'.jpg'
    per_gallery=cv2.imread(per_gallery_name)
    # Image pre-processing
    per_gallery = cv2.GaussianBlur(per_gallery,(3,3),0)
    per_gallery = cv2.cvtColor(per_gallery,cv2.COLOR_BGR2YUV)
    per_gallery[:,:,0] = cv2.equalizeHist(per_gallery[:,:,0])
    per_gallery = cv2.cvtColor(per_gallery,cv2.COLOR_YUV2BGR)
    # feature extraction for per gallery
    per_gallery_kp, per_gallery_des = sift.detectAndCompute(per_gallery,None)
    gallery_kp.append(per_gallery_kp)
    gallery_des.append(per_gallery_des)

In [ ]:
for i, query_img_no in tqdm(enumerate(query_imgs_no[0:20])):
    time_s = time.time()
    dist_record=[]
    per_query_name=path_query+'/'+str(query_img_no)+'.jpg'
    per_query=cv2.imread(per_query_name)
    gallery_imgs_no_desc=[]
    
    # read boundary from text file
    queryfilename = path_query_txt+'/'+str(query_img_no)+'.txt'
    boundary_file = open(queryfilename, 'r')
    boundary = boundary_file.readline().strip().split(' ')
    boundary = [int(b) for b in boundary]
    boundary_file.close()
    
    # crop the image
    x ,y, w, h = boundary
    query_boundary = per_query[y:y+h, x:x+w]

    # Image pre-processing
    query_boundary = cv2.GaussianBlur(query_boundary,(3,3),0)
    query_boundary=cv2.cvtColor(query_boundary,cv2.COLOR_BGR2YUV)
    query_boundary[:,:,0] = cv2.equalizeHist(query_boundary[:,:,0])
    query_boundary = cv2.cvtColor(query_boundary,cv2.COLOR_YUV2BGR)
    # feature extraction for per query
    per_query_kp, per_query_des = sift.detectAndCompute(query_boundary,None)
    # the iteration loop for gallery
    for j, gallery_img_no in tqdm(enumerate(gallery_imgs_no), desc=f"Processing query part {i}"):
        matches = bf.knnMatch(per_query_des.astype(np.float32),gallery_des[j].astype(np.float32),k=2)
        # Compute similarity score
        sim_score = cosine_similarity(per_query_des, gallery_des[j])
        dist_record.append(sim_score)
        # find the indexes with descending similarity order
    ascend_index=sorted(range(len(dist_record)), key=lambda k: np.max(dist_record[k]),reverse=True)
    # update the results for one query
    for k in range(len(ascend_index)):
        gallery_imgs_no_desc.append(np.array(gallery_imgs_no)[ascend_index[k]])
    record_all[i,:]= gallery_imgs_no_desc
    time_e = time.time()
    print('retrieval time for query {} is {}s'.format(query_img_no, time_e-time_s))
    query_idx = i
    print(f'For query image No. {query_imgs_no[query_idx]}, the top ranked similar image No. is {gallery_imgs_no_desc[0]}.')

In [ ]:
# write the output file following the example
f=open(r'./rank_list_SIFT.txt','w')
for i in range(num_query):
    f.write('Q'+str(i+1)+': ')
    for j in range(len(name_gallery)):
        f.write(str(np.int32(record_all[i,j]))+' ')
    f.write('\n')
f.close()